In [12]:
# 네비게이션 링크 추출 --> 리스트로
# 글 링크 추출 --> 리스트로
# 최종 중복링크 제거된 리스트 확보 목적
import requests
from bs4 import BeautifulSoup

url = "http://ssacademy.dothome.co.kr/01.html"
response = requests.get(url)
response.encoding = 'utf-8'
html = response.text
soup = BeautifulSoup(html, 'html.parser')

navi = soup.select('a.nav-link')

article = soup.select('a.article-link')

full_list = list(set([a['href'] for a in navi] + [a['href'] for a in article]))

print(full_list)

domain = 'example.com'
for link in full_list:
    if domain in link:
        print(f"{link} : {domain}이 포함됨")
    else:
        print(f"{link} : {domain}이 포함하지 않음")

['https://example.com/contact', 'https://example.com/article/3', 'https://example.com/about', 'https://example.com/home', 'https://example.com/article/1', 'https://example.com/article/2']
https://example.com/contact : example.com이 포함됨
https://example.com/article/3 : example.com이 포함됨
https://example.com/about : example.com이 포함됨
https://example.com/home : example.com이 포함됨
https://example.com/article/1 : example.com이 포함됨
https://example.com/article/2 : example.com이 포함됨


In [44]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
url = "http://ssacademy.dothome.co.kr/02.html"

response = requests.get(url)
response.encoding = 'utf-8'
html = response.text
soup = BeautifulSoup(html, 'html.parser')

table = soup.select_one('table.data-table')
column = [ th.text.strip() for th in soup.select('thead th')]

data = []
rows = soup.select("tbody tr.student-row")
for row in rows:
    data.append([int(td.text) if td.text.isdigit() else td.text for td in row.select('td')])

# print(trs)
print(column)
print(data)

df = pd.DataFrame(data, columns = column)
# df.info()
df.to_csv("student_score.csv", index = False, encoding = 'utf-8')
df

['순위', '이름', '점수', '등급']
[[1, '김철수', 95, 'A+'], [2, '이영희', 88, 'B+']]


,순위,이름,점수,등급
0,1,김철수,95,A+
1,2,이영희,88,B+


In [1]:
# DB에 값 넣기
import requests
from bs4 import BeautifulSoup
import sqlite3

url = "http://ssacademy.dothome.co.kr/02.html"

response = requests.get(url)
response.encoding = 'utf-8'
soup = BeautifulSoup(response.text, "html.parser")

columns = []
for th in soup.select("thead th"):
    columns.append(th.text)

data = []
rows = soup.select("tbody tr.student-row")
for row in rows:
    
    row_data = []
    for td in row.select("td"):
        row_data.append(int(td.text) if td.text.isdigit() else td.text)
    
    data.append(row_data)

conn = sqlite3.connect('student_scores.db')

create_table_query = """
CREATE TABLE IF NOT EXISTS StudentScores (
    Rank INTEGER,
    Name TEXT,
    Score INTEGER,
    Grade TEXT
);
"""
conn.execute(create_table_query)

insert_query = "INSERT INTO StudentScores (Rank, Name, Score, Grade) VALUES (?, ?, ?, ?)"
# conn.executemany(insert_query, data)
for row in data:
    conn.execute(insert_query, row)

conn.commit()
conn.close()

print(columns)
print(data)


['순위', '이름', '점수', '등급']
[[1, '김철수', 95, 'A+'], [2, '이영희', 88, 'B+']]


In [ ]:
import requests
from bs4 import BeautifulSoup

url = "http://ssacademy.dothome.co.kr/03.html"

response = requests.get(url)
response.encoding = 'utf-8'
soup = BeautifulSoup(response.text, "html.parser")

form_data = {}

inputs = soup.select('input[type = "text"], input[type = "email"]')
for input in inputs:
    # print(input['value'])
    value = input.get('value','')
    form_data[input['name']] = value

select_opt = soup.select_one('select > option[selected]')
# select_opt['value']
select_opt_value = select_opt.get('value','')
print(select_opt.text)
select_ele = select_opt.find_parent('select')
form_data[select_ele['name']] = select_opt_value

checkbox = soup.select_one("input[type = 'checkbox']")
checked = checkbox.has_attr('checked')
form_data[checkbox['name']] = checked

form_data

과학


{'username': 'test_user',
 'email': 'test@example.com',
 'interests': 'science',
 'subscribe': True}

In [ ]:
import requests
from bs4 import BeautifulSoup

url = "http://ssacademy.dothome.co.kr/04.html"

response = requests.get(url)
response.encoding = 'utf-8'
soup = BeautifulSoup(response.text, "html.parser")

select_opts = soup.select('select > option[selected]')

if select_opts:
    select_ele = select_opts[0].find_parent('select')
    form_data2 = {select_ele['name'] : []}
    for select_opt in select_opts:
        # if select_ele['name'] in form_data2:
        #     form_data2[select_ele['name']].append(select_opt.text)
        # else:
        #     form_data2[select_ele['name']] = []
        #     form_data2[select_ele['name']].append(select_opt.text)
        form_data2[select_ele['name']].append(select_opt.text)
else:
    print("선택된 값이 없습니다")
form_data2


{'fruits': ['바나나', '체리']}

In [3]:
# [
# {'id': 1, 'src': 'image1.png', 'alt': '첫 번째 이미지', 'title': '아름다운 풍경', 'photographer': '김작가'}, 
# {'id': 2, 'src': 'image2.png', 'alt': '두 번째 이미지', 'title': '도시의, 밤', 'photographer': '이작가'}
#]

# 이미지는 img폴더에 저장
# 메타데이터는 DB로 저장

import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import os
import sqlite3

url = "http://ssacademy.dothome.co.kr/05.html"

response = requests.get(url)
response.encoding = 'utf-8'
soup = BeautifulSoup(response.text, "html.parser")

images = []

for container in soup.select('div.image-container'):
    img = container.select_one('img')
    info = container.select_one('.image-info')

    image_data = {
        'id': int(img['data-id']),
        'src': img['src'],
        'alt': img['alt'],
        'title': info.select_one(".title").text,
        'photographer': info.select_one(".photographer").text.split(": ")[-1]
    }

    images.append(image_data)

print(images)


conn = sqlite3.connect('images.db')
cursor = conn.cursor()
cursor.execute('''
    CREATE TABLE IF NOT EXISTS images (
        id INTEGER PRIMARY KEY,
        src TEXT,
        alt TEXT,
        title TEXT,
        photographer TEXT
    )
''')
conn.commit()


os.makedirs('img', exist_ok=True)  

for image in images:
    image_url = urljoin(url, image['src'])
    save_path = "img/" + image['src']
    response = requests.get(image_url)
    if response.status_code == 200:
        with open(save_path, 'wb') as f:
            f.write(response.content)
        cursor.execute('''
            INSERT INTO images (id, src, alt, title, photographer)
            VALUES (?, ?, ?, ?, ?)
        ''', (image['id'], image['src'], image['alt'], image['title'], image['photographer']))

conn.commit()
conn.close()

[{'id': 1, 'src': 'image1.png', 'alt': '첫 번째 이미지', 'title': '아름다운 풍경', 'photographer': '김작가'}, {'id': 2, 'src': 'image2.png', 'alt': '두 번째 이미지', 'title': '도시의, 밤', 'photographer': '이작가'}]


In [30]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

url = "http://ssacademy.dothome.co.kr/06.html"

html = requests.get(url)

soup = BeautifulSoup(html.text,'html.parser')

articles = soup.select('article.post-wrapper')

blogs = []

for article in articles:
    # 굵은 클래스 가져오기
    header = article.select_one('div.post-header')
    content = article.select_one('div.content')
    tags = article.select_one('ul.tags')

    # 필요한 태그의 클래스명(키값) 가져오기
    title_key = header.select_one('h1.title')['class'][0]
    date_key = header.select_one('span.date')['class'][0]
    author_key = header.select_one('div.author')['class'][0]
    uid_key = 'data-userid'
    content_key = content['class'][0]
    tags_key = tags['class'][0]

    # 가져온 키값들을 리스트로 저장
    keys = [title_key, date_key, author_key, uid_key, content_key, tags_key]
    # print(keys)

    # 필요한 태그의 값 가져오기
    title = header.select_one('h1.title').text
    date = header.select_one('span.date').text
    author = header.select_one('div.author').text.split(':')[-1].strip()
    uid = header.select_one('div.author')['data-userid']
    content = content.text.strip()
    tag_list = [tag.text for tag in tags.select('li')]

    # 값들을 리스트로 저장
    values = [title, date, author, uid, content, tag_list]
    # print(values)

    # 키와 값을 딕셔너리로 변환해 리스트에 저장
    blogs.append(dict(zip(keys,values)))

blogs


[{'title': '블로그 제목 1',
  'date': '2024-01-01',
  'author': '김블로그',
  'data-userid': 'user123',
  'content': '첫 번째 블로그 내용입니다...\n추가 내용...',
  'tags': ['#python', '#coding']},
 {'title': '블로그 제목 2',
  'date': '2024-01-03',
  'author': '이블로그',
  'data-userid': 'user456',
  'content': '오늘 엄청나게 추워요\n손도 시렵고 발도 시렵고 귀도 시려워요\n모두 힘내요!',
  'tags': ['#crawling', '#coding']},
 {'title': '블로그 제목 3',
  'date': '2025-08-03',
  'author': '박블로그',
  'data-userid': 'user789',
  'content': '오늘 너무 덥네요\n교실이 최고!!! 나가지 맙시다~~~\n모두 힘내요!',
  'tags': ['#ai', '#project']}]